In [ ]:
# Parsing SR

In [ ]:
!pip install pypdf2 docling -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/54

In [ ]:
import os
os.environ["TORCH_CUDA_ARCH_LIST"] = "7.5"

In [ ]:
from google.colab import files

uploaded = files.upload()
for file_name in uploaded.keys():
    print(f"File diunggah: {file_name}")

Saving asii_2023_sustainability_report.pdf to asii_2023_sustainability_report.pdf
File diunggah: asii_2023_sustainability_report.pdf


In [ ]:
from typing import List, Dict, Any
import pandas as pd
import PyPDF2
import glob
from docling.document_converter import DocumentConverter
from tqdm import tqdm
import json
import os
import re
import uuid
from pydantic import BaseModel

class PreprocessorRequest(BaseModel):
    file_path: str
    company: str
    ticker: str
    year: int

class PreprocessorResponse:
    def __init__(self, results: List[Dict[str, Any]]):
        self.results = results

    def save_to_json(self, output_file_path: str):
        try:
            with open(output_file_path, 'w') as json_file:
                json.dump(self.results, json_file, indent=4)
                print(f"Results saved to {output_file_path}")
        except Exception as e:
            print(f"An error occurred while saving to JSON: {e}")

class Preprocessor:
    def __init__(self):
        self.valid_indicators = [
            '2-1', '2-2', '2-3', '2-4', '2-5', '2-6', '2-7', '2-8', '2-9', '2-10',
            '2-11', '2-12', '2-13', '2-14', '2-15', '2-16', '2-17', '2-18', '2-19',
            '2-20', '2-21', '2-22', '2-23', '2-24', '2-25', '2-26', '2-27', '2-28',
            '2-29', '2-30', '3-1', '3-2', '3-3', '201-1', '201-2', '201-3', '201-4',
            '202-1', '202-2', '203-1', '203-2', '204-1', '205-1', '205-2', '205-3',
            '206-1', '207-1', '207-2', '207-3', '207-4', '301-1', '301-2', '301-3',
            '302-1', '302-2', '302-3', '302-4', '302-5', '303-1', '303-2', '303-3',
            '303-5', '304-1', '304-2', '304-3', '304-4', '305-1', '305-2', '305-3',
            '305-4', '305-5', '305-6', '305-7', '306-1', '306-2', '306-3', '306-4',
            '306-5', '308-1', '308-2', '401-1', '401-2', '401-3', '402-1', '403-1',
            '403-2', '403-3', '403-4', '403-5', '403-6', '403-7', '403-8', '403-9',
            '403-10', '404-1', '404-2', '404-3', '405-1', '405-2', '406-1', '407-1',
            '408-1', '409-1', '410-1', '411-1', '413-1', '413-2', '414-1', '414-2',
            '415-1', '416-1', '416-2', '417-1', '417-2', '417-3', '418-1'
        ]

    def get_pdf_page_count(self, pdf_path: str) -> int:
        try:
            with open(pdf_path, 'rb') as pdf_file:
                reader = PyPDF2.PdfReader(pdf_file)
                return len(reader.pages)
        except Exception as e:
            print(f"An error occurred while counting pages: {e}")
            return -1

    def extract_page(self, input_pdf_path: str, output_pdf_path: str, page_number: int):
        try:
            with open(input_pdf_path, 'rb') as input_pdf:
                reader = PyPDF2.PdfReader(input_pdf)
                writer = PyPDF2.PdfWriter()

                if page_number < 0 or page_number >= len(reader.pages):
                    raise ValueError("Invalid page number.")

                writer.add_page(reader.pages[page_number])

                with open(output_pdf_path, 'wb') as output_pdf:
                    writer.write(output_pdf)
        except Exception as e:
            print(f"An error occurred: {e}")

    def get_indicator(self, content: str) -> List[str]:
        pattern = r'\b\d{1,3}-\d{1,2}(?=\D|$)'
        found_codes = re.findall(pattern, content)
        disclosed_indicators = [code for code in found_codes if code in self.valid_indicators]
        disclosed_indicators = list(set(disclosed_indicators))
        return disclosed_indicators

    def process(self, request: PreprocessorRequest) -> PreprocessorResponse:
        file_path = request.file_path
        company = request.company
        ticker = request.ticker
        year = request.year

        page_count = self.get_pdf_page_count(file_path)
        if page_count == -1:
            raise ValueError("Failed to get the page count of the PDF.")

        results = []
        for i in tqdm(range(page_count)):
            temp_pdf_path = "temp.pdf"
            self.extract_page(file_path, temp_pdf_path, i)

            converter = DocumentConverter()
            result = converter.convert(temp_pdf_path)
            result_md = result.document.export_to_markdown()
            disclosed_gri = self.get_indicator(result_md)

            results.append({
                "page_content": result_md,
                "metadata": {
                    "page": i + 1,
                    "company": company,
                    "year": year,
                    "ticker": ticker,
                    "disclosed_gri": disclosed_gri
                },
                "id": str(uuid.uuid4())
            })

        return PreprocessorResponse(results)


In [ ]:
file = "asii_2023_sustainability_report.pdf"
company = "PT Astra International Tbk"
ticker = "ASII"
year = "year"

# Buat instance preprocessor
preprocessor = Preprocessor()

# Buat request untuk preprocessing
request = PreprocessorRequest(file_path=file, company=company, ticker=ticker, year=2022)

# Proses file dengan preprocessor
try:
    response = preprocessor.process(request)

    # Simpan hasilnya ke file JSON
    output_json = os.path.basename(file)[:-4] + '_output.json'
    response.save_to_json(output_json)
    print(f"Hasil preprocessing telah disimpan di {output_json}")
except Exception as e:
    print(f"Terjadi kesalahan selama preprocessing: {e}")

100%|██████████| 210/210 [18:22<00:00,  5.25s/it]

Results saved to asii_2023_sustainability_report_output.json
Hasil preprocessing telah disimpan di asii_2023_sustainability_report_output.json
